# Comparing KN surrogate models

**Abstract:** Here we compare the MSE of the surrogate models between, e.g. Tensorflow and Jax/flax models.

In [1]:
%load_ext autoreload 
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from nmma.em.training import SVDTrainingModel
import nmma as nmma
import time
import arviz

params = {"axes.grid": True,
        "text.usetex" : True,
        "font.family" : "serif",
        "ytick.color" : "black",
        "xtick.color" : "black",
        "axes.labelcolor" : "black",
        "axes.edgecolor" : "black",
        "font.serif" : ["Computer Modern Serif"],
        "xtick.labelsize": 16,
        "ytick.labelsize": 16,
        "axes.labelsize": 16,
        "legend.fontsize": 16,
        "legend.title_fontsize": 16,
        "figure.titlesize": 16}

plt.rcParams.update(params)

from nmma.em.io import read_photometry_files
from nmma.em.utils import interpolate_nans

import inspect 
import nmma.em.model_parameters as model_parameters

MODEL_FUNCTIONS = {
    k: v for k, v in model_parameters.__dict__.items() if inspect.isfunction(v)
}

model_name = "Bu2022Ye"
model_function = MODEL_FUNCTIONS[model_name]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/urash/twouters/miniconda3/envs/nmma_gpu/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/urash/twouters/miniconda3/envs/nmma_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Install afterglowpy if you want to simulate afterglows.
Install wrapt_timeout_decorator if you want timeout simulations.


In [2]:
import jax
import jaxlib
jax.devices() # check if CUDA is present

[cuda(id=0)]

## Preprocessing data

In [3]:
lcs_dir = "/home/urash/twouters/KN_Lightcurves/lightcurves/lcs_bulla_2022" # for remote SSH Potsdam
out_dir = "/home/urash/twouters/nmma_models/flax_models/" # initial flax models will be saved here
filenames = os.listdir(lcs_dir)
full_filenames = [os.path.join(lcs_dir, f) for f in filenames]
print(f"There are {len(full_filenames)} lightcurves for this model.")

There are 7700 lightcurves for this model.


In [4]:
# Read the data and interpolate the NaNs, and get training data
data = read_photometry_files(full_filenames)
data = interpolate_nans(data)
training_data, parameters = model_function(data)

In [5]:
#Extract times and filters
key = list(training_data.keys())[0]
example = training_data[key]
t = example["t"]
keys = list(example.keys())
filts = [k for k in keys if k not in parameters + ["t"]]

## Get the flax model


In [6]:
training_model = SVDTrainingModel(
        model_name,
        training_data,
        parameters,
        t,
        filts,
        interpolation_type="flax",
        svd_path=out_dir # initial flax models will be saved here
    )

print(training_model.svd_path)

The grid will be interpolated to sample_time with interp1d
Model exists... will load that model.
/home/urash/twouters/nmma_models/flax_models/


In [7]:
training_model.__dict__.keys()

dict_keys(['model', 'data', 'model_parameters', 'sample_times', 'filters', 'n_coeff', 'n_epochs', 'interpolation_type', 'data_type', 'data_time_unit', 'plot', 'plotdir', 'ncpus', 'univariate_spline', 'univariate_spline_s', 'random_seed', 'svd_path', 'svd_model'])

## Get the Tensorflow model (to do)

In [8]:
# Read the data and interpolate the NaNs, and get training data
data = read_photometry_files(full_filenames)
data = interpolate_nans(data)
training_data, parameters = model_function(data)

KeyboardInterrupt: 

In [ ]:
svd_path_tensorflow = "/home/urash/twouters/nmma_models"
tensorflow_model = SVDTrainingModel(
        model_name,
        training_data,
        parameters,
        t,
        filts,
        interpolation_type="tensorflow",
        svd_path=svd_path_tensorflow
    )

The grid will be interpolated to sample_time with interp1d
Model exists... will load that model.


2023-12-12 15:32:57.704761: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 15:32:57.704856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 15:32:57.704930: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 15:32:58.875317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/home/urash/twouters/nmma_models/flax_models/


## Get the input and output pairs of the Bu2022Ye model

In [ ]:
def get_input_values(training_data: dict, parameters: list) -> np.ndarray:
    """
    From a dictionary of training data, extract the input values for the model.
    """
    
    input_values = []
    
    for key in training_data.keys():
        data = training_data[key]
        input_values.append([data[param] for param in parameters])
    
    return np.array(input_values)

In [ ]:
def get_output_values(training_data: dict) -> np.ndarray:
    """
    From a dictionary of training data, extract the output values for the model.
    """
    
    output_values = [training_data[key]["data"] for key in training_data.keys()]
    
    return np.array(output_values)

In [ ]:
input_values = get_input_values(training_data, parameters)
output_values = get_output_values(training_data)

In [ ]:
print(np.shape(input_values))
print(np.shape(output_values))

(7700, 6)
(7700, 100, 26)


NOTE: this can be done faster with jax.vmap etc, but not for 

In [ ]:
N = 100
# Select a random subset of indices for the input values
idx_list = np.random.choice(len(input_values), N, replace=False)
sampled_input_values = input_values[idx_list]
sampled_output_values = output_values[idx_list]

Get the flax output

In [ ]:
# For this list, we compute the LCs using the flax model
flax_output = []
start = time.time()
for i in idx_list:
    # Compute the lightcurve
    _, _, mag = nmma.em.utils.calc_lc(t,
                                input_values[i], 
                                svd_mag_model = training_model.svd_model, 
                                interpolation_type="flax", 
                                filters = filts, 
                                )
    # Convert this dictionary to values of the LCs
    mag = mag.values()
    mag = np.array(list(mag)).T
    flax_output.append(mag)
end = time.time()
print(f"Computing all the flax lightcurves for a subset of {N} lightcurves took {end-start} seconds.")
# Make sure this is a np.ndarray
flax_output = np.array(flax_output)

Install afterglowpy if you want to simulate afterglows.
Install wrapt_timeout_decorator if you want timeout simulations.


TypeError: Expected a callable value, got (array([ 0.103,  0.109,  0.116,  0.122,  0.13 ,  0.137,  0.145,  0.154,
        0.163,  0.173,  0.183,  0.194,  0.206,  0.218,  0.231,  0.244,
        0.259,  0.274,  0.29 ,  0.308,  0.326,  0.345,  0.366,  0.387,
        0.41 ,  0.435,  0.461,  0.488,  0.517,  0.547,  0.58 ,  0.614,
        0.651,  0.689,  0.73 ,  0.773,  0.819,  0.868,  0.919,  0.974,
        1.032,  1.093,  1.158,  1.226,  1.299,  1.376,  1.458,  1.544,
        1.636,  1.733,  1.836,  1.944,  2.06 ,  2.182,  2.311,  2.448,
        2.594,  2.747,  2.91 ,  3.083,  3.266,  3.459,  3.665,  3.882,
        4.112,  4.356,  4.614,  4.888,  5.178,  5.485,  5.81 ,  6.155,
        6.52 ,  6.906,  7.316,  7.75 ,  8.21 ,  8.696,  9.212,  9.759,
       10.337, 10.95 , 11.6  , 12.288, 13.016, 13.788, 14.606, 15.472,
       16.39 , 17.362, 18.392, 19.482, 20.638, 21.862, 23.158, 24.532,
       25.987, 27.528, 29.16 , 30.89 ]), array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, in

Get the TF output

In [ ]:
# For this list, we compute the LCs using the flax model
tf_output = []
start = time.time()
for i in idx_list:
    # Compute the lightcurve
    _, _, mag = nmma.em.utils.calc_lc(t,
                                input_values[i], 
                                svd_mag_model = tensorflow_model.svd_model, 
                                interpolation_type="tensorflow", 
                                filters = filts, 
                                )
    # Convert this dictionary to values of the LCs
    mag = mag.values()
    mag = np.array(list(mag)).T
    tf_output.append(mag)
end = time.time()
print(f"Computing all the tensorflow lightcurves for a subset of {N} lightcurves took {end-start} seconds.")
# Make sure this is a np.ndarray
tf_output = np.array(tf_output)

Computing all the flax lightcurves for a subset of 100 lightcurves took 5.414904594421387 seconds.


## Compare MSE or MAE values

TODO: Best to compare this as a distribution, and perhaps best to consider MAE, or some self-defined loss function or error function?

In [ ]:
def mse(y_true, y_pred, axis=None):
    return np.mean((y_true - y_pred)**2, axis=axis)

def se(y_true, y_pred):
    return (y_true - y_pred)**2

def mae(y_true, y_pred, axis=None):
    return np.mean(np.abs(y_true - y_pred), axis=axis)

def ae(y_true, y_pred):
    return np.abs(y_true - y_pred)

def my_format(low: float, med: float, high: float, nb: int = 3) -> str:
    med = np.round(med, nb)
    low = med - low
    low = np.round(low, nb)
    high = high - med
    high = np.round(high, nb)
    
    return f"{med} - {low} + {high}"

# # TODO with arviz summarize the errors
# def summarize_data(values: np.array, percentile: float = 0.95) -> None:
    
#     med = np.median(values)
#     result = arviz.hdi(values, hdi_prob = percentile)
    
#     print(my_format(low, med, high))
    
#     return

In [ ]:
which_dataset = flax_output
which_error = mae

diffs = se(which_dataset, sampled_output_values)
axis = 0
mse_values = which_error(which_dataset, sampled_output_values, axis=axis)
mse_values = np.mean(mse_values, axis=0)
for f, val in zip(filts, mse_values):
    print(f"{f}: {val}")

bessellux: 1.860606712292868
bessellb: 1.7921464100782574
bessellv: 1.2305521863871158
bessellr: 0.8128799439449892
besselli: 0.7583155514392446
sdssu: 1.0992091068356826
ps1__g: 1.8798792498944232
ps1__r: 1.0871671955916962
ps1__i: 0.8739246164214045
ps1__z: 0.8943951113252402
ps1__y: 0.9373291937542888
uvot__b: 1.8021596713178516
uvot__u: 1.7665092726441651
uvot__uvm2: 1.871222932129877
uvot__uvw1: 1.6923179907486763
uvot__uvw2: 1.991923127090116
uvot__v: 1.5010415651636924
uvot__white: 0.8017472904174968
atlasc: 1.278656720359771
atlaso: 0.7374362913587105
2massj: 0.7798304492014924
2massh: 0.5952465996901724
2massks: 0.679392397537318
ztfg: 1.8178616188139933
ztfr: 1.0294186804296865
ztfi: 0.8224096511046391


In [ ]:
print("MAE flax")
print(mae(flax_output, sampled_output_values))
print("MSE flax")
print(mse(flax_output, sampled_output_values))

MAE flax
1.2459069052297256
MSE flax
20.490649731915262


In [ ]:
print("MAE tf")
print(mae(tf_output, sampled_output_values))
print("MSE tf")
print(mse(tf_output, sampled_output_values))

MAE tf
1.958473016121413
MSE tf
44.68970564439654


## Speed: can we improve the speed generation of flax?

In [ ]:
import jax
import jax.numpy as jnp
import scipy.interpolate as interp

In [ ]:
# def calc_lc_flax(
#     tt,
#     param_list, # TODO add type hint, but can break if jax not imported
#     svd_mag_model=None,
#     svd_lbol_model=None,
#     mag_ncoeff=None,
#     lbol_ncoeff=None,
#     filters=None,
#     use_jit=True, # TODO implement a jitted version?
# ):
#     mAB = {}

#     if filters is None:
#         filters = list(svd_mag_model.keys())
#     else:
#         # add null output for radio and X-ray filters
#         for filt in filters:
#             if filt.startswith(("radio", "X-ray")):
#                 mAB[filt] = jnp.inf * jnp.ones(len(tt))

#     for jj, filt in enumerate(filters):
#         if filt in mAB:
#             continue

#         if mag_ncoeff:
#             n_coeff = min(mag_ncoeff, svd_mag_model[filt]["n_coeff"])
#         else:
#             n_coeff = svd_mag_model[filt]["n_coeff"]
#         # param_array = svd_mag_model[filt]["param_array"]
#         # cAmat = svd_mag_model[filt]["cAmat"]
#         VA = svd_mag_model[filt]["VA"]
#         param_mins = svd_mag_model[filt]["param_mins"]
#         param_maxs = svd_mag_model[filt]["param_maxs"]
#         mins = svd_mag_model[filt]["mins"]
#         maxs = svd_mag_model[filt]["maxs"]
#         tt_interp = svd_mag_model[filt]["tt"]

#         param_list_postprocess = (param_list - param_mins) / (param_maxs - param_mins)

#         # Watch out for possible confusion with names here
#         state = svd_mag_model[filt]["model"]

#         # Apply the model to the given parameters
#         cAproj = state.apply_fn({'params': state.params}, param_list_postprocess)
#         cAstd = jnp.ones((n_coeff,))

#         # Go from SVD coefficients to original lightcurve data
#         mag_back = jnp.dot(VA[:, :n_coeff], cAproj)
#         mag_back = mag_back * (maxs - mins) + mins

#         ## TODO how to implement this in jax?
#         # ii = jnp.where((~jnp.isnan(mag_back)) * (tt_interp < 20.0))[0]
#         # if len(ii) < 2:
#         #     maginterp = jnp.nan * jnp.ones(tt.shape)
#         # else:
#         #     f = interp.interp1d(tt_interp[ii], mag_back[ii], fill_value="extrapolate")
#         #     maginterp = f(tt)

#         # f = interp.interp1d(tt, mag_back, fill_value="extrapolate")
#         # maginterp = f(tt)

#         maginterp = mag_back
#         mAB[filt] = maginterp

#     # TODO: currently not used, what if we want to use it?
#     if svd_lbol_model is not None:
#         if lbol_ncoeff:
#             n_coeff = min(lbol_ncoeff, svd_lbol_model["n_coeff"])
#         else:
#             n_coeff = svd_lbol_model["n_coeff"]
#         # param_array = svd_lbol_model["param_array"]
#         # cAmat = svd_lbol_model["cAmat"]
#         VA = svd_lbol_model["VA"]
#         param_mins = svd_lbol_model["param_mins"]
#         param_maxs = svd_lbol_model["param_maxs"]
#         mins = svd_lbol_model["mins"]
#         maxs = svd_lbol_model["maxs"]
#         gps = svd_lbol_model["gps"]
#         tt_interp = svd_lbol_model["tt"]

#         param_list_postprocess = param_list
#         for i in range(len(param_mins)):
#             param_list_postprocess.at[i].set((param_list_postprocess[i] - param_mins[i]) / (
#                 param_maxs[i] - param_mins[i]
#             ))

#         # TODO add this for flax?
#         # ...
#         # ...

#         lbol_back = jnp.dot(VA[:, :n_coeff], cAproj)
#         lbol_back = lbol_back * (maxs - mins) + mins
#         # lbol_back = scipy.signal.medfilt(lbol_back, kernel_size=3)

#         ii = jnp.where(~jnp.isnan(lbol_back))[0]
#         if len(ii) < 2:
#             lbolinterp = jnp.nan * jnp.ones(tt.shape)
#         else:
#             f = interp.interp1d(tt_interp[ii], lbol_back[ii], fill_value="extrapolate")
#             lbolinterp = 10 ** f(tt)
#         lbol = lbolinterp
#     else:
#         lbol = np.inf * np.ones(len(tt))

#     return np.squeeze(tt), np.squeeze(lbol), mAB

### 1. Jit, no vmap

In [ ]:
# Lambda function, so that we focus on the parameters as being the input only
from nmma.em.utils import get_calc_lc_jit
calc_lc_given_params_jit = get_calc_lc_jit(t, svd_mag_model=training_model.svd_model, filters=filts)
start = time.time()
for i in range(N):
    flax_output_jit = calc_lc_given_params_jit(sampled_input_values[i])
end = time.time()
print(f"Computing all the flax lightcurves for a subset of {N} lightcurves took {end-start} seconds.")

Install afterglowpy if you want to simulate afterglows.
Install wrapt_timeout_decorator if you want timeout simulations.
Computing all the flax lightcurves for a subset of 100 lightcurves took 1.4477312564849854 seconds.


### 2. Jit and vmap

In [ ]:
# vmap the function
calc_lc_given_params_vmap = jax.vmap(calc_lc_given_params_jit)
# apply to input_values
flax_output_jit = calc_lc_given_params_vmap(sampled_input_values)